In [ ]:
# ETL Complaint Facts
# If using the native Google BigQuery API module:
from google.cloud import bigquery
from google.cloud.exceptions import NotFound
import pandas as pd
import os
from datetime import datetime
from google.oauth2 import service_account

In [ ]:
# Set the GCP Project, dataset and table name
gcp_project = 'cis-4400-404715'
path_to_service_account_key_file = 'keys.json'

In [ ]:
def upload_bigquery_table(bqclient, table_path, write_disposition, df):
    """
    upload_bigquery_table
    Accepts a path to a BigQuery table, the write disposition and a dataframe
    Loads the data into the BigQuery table from the dataframe.
    for credentials.
    The write disposition is either
    write_disposition="WRITE_TRUNCATE"  Erase the target data and load all new data.
    write_disposition="WRITE_APPEND"    Append to the existing table
    """
    try:
        
        job_config = bigquery.LoadJobConfig(write_disposition=write_disposition)
        
        # Submit the job
        job = bqclient.load_table_from_dataframe(df, table_path, job_config=job_config)
        
        # Show the job results
        job.result()
    except Exception as err:
        print("Failed to load BigQuery Table.", err)
        # os._exit(-1)

In [ ]:
def bigquery_table_exists(table_path, bqclient):
    """
    bigquery_table_exists
    Accepts a path to a BigQuery table
    Checks if the BigQuery table exists.
    Returns True or False
    """
    try:
        bqclient.get_table(table_path)  # Make an API request.
        return True
    except NotFound:
        # print("Table {} is not found.".format(table_id))
        return False

In [ ]:
def build_new_table(bqclient, table_path, df):
    """
    build_new_table
    Accepts a path to a dimensional table, the dimension name and a data frame
    Add the surrogate key and a record timestamp to the data frame
    Inserts the contents of the dataframe to the dimensional table.
    """
    upload_bigquery_table(bqclient, table_path, "WRITE_TRUNCATE", df)

In [ ]:
def insert_existing_table( bqclient, table_path, df):
    """
    insert_existing_table
    Accepts a path to a dimensional table, the dimension name and a data frame
    Compares the new data to the existing data in the table.
    Inserts the new/modified records to the existing table
    """
    upload_bigquery_table( bqclient, table_path, "WRITE_APPEND", df)

In [ ]:
def query_bigquery_table(table_path, bqclient, surrogate_key):
    """
    query_bigquery_table
    Accepts a path to a BigQuery table and the name of the surrogate key
    Queries the BigQuery table but leaves out the update_timestamp and surrogate key columns
    Returns the dataframe
    """    
    bq_df = pd.DataFrame
    # sql_query = 'SELECT * EXCEPT ( update_timestamp, '+surrogate_key+') FROM `' + table_path + '`'
    sql_query = 'SELECT * FROM `' + table_path + '`'
    bq_df = bqclient.query(sql_query).to_dataframe()
    return bq_df

In [ ]:
def dimension_lookup(dimension_name, lookup_columns, df):
    """
    dimension_lookup
    Lookup the lookup_columns in the dimension_name and return the associated surrogate keys
    Returns dataframe augmented with the surrogate keys
    """
    bq_df = pd.DataFrame
    surrogate_key = dimension_name+"_dim_id"
    dimension_table_path = ".".join([gcp_project,bq_dataset,dimension_name+"_dimension"])
    # Fetch the existing table
    bq_df = query_bigquery_table(dimension_table_path, bqclient, surrogate_key)
    if dimension_name == 'date':
        bq_df['full_date'] = bq_df['full_date'].apply(lambda x: x.strftime('%Y-%m-%d'))

    print(bq_df)
    # Melt the dimension dataframe into an index with the lookup columns
    m = bq_df.melt(id_vars=lookup_columns, value_vars=surrogate_key)
    print(m)
    # Rename the "value" column to the surrogate key column name
    m=m.rename(columns={"value":surrogate_key})
    # Merge with the fact table record
    df = df.merge(m, on=lookup_columns, how='left')
    # Drop the "variable" column and the lookup columns
    df = df.drop(columns=lookup_columns)
    df = df.drop(columns="variable")
    #print(df)
    return df

In [ ]:
def rename_column(df, bq_dataset, dimension_name):

    # Renaming for 311
    if bq_dataset == '311_illegal_parking':
        if dimension_name == 'complaint':
            df = df.rename(columns={'descriptor': 'complaint_description'})
        elif dimension_name == 'complaint_source':
            df = df.rename(columns={'open_data_channel_type': 'complaint_source_channel'})
        elif dimension_name == 'location':
            df = df.rename(columns={'city': 'incident_city', 'incident_zip': 'incident_zipcode'})
        elif dimension_name == 'date':
            df = df.rename(columns={'created_date': 'full_date'})

    # Renaming for Open Parking
    elif bq_dataset == 'open_parking':
        if dimension_name == 'agency':
            df = df.rename(columns={'issuing_agency': 'agency_name'})
        elif dimension_name == 'date':
            df = df.rename(columns={'issue_date': 'full_date'})
        elif dimension_name == 'location':
            df = df.rename(columns={'precinct': 'precinct_num', 'county': 'borough', 'zipcode': 'incident_zipcode'})
        elif dimension_name == 'violation':
            df = df.rename(columns={'violation': 'violation_description'})
        elif dimension_name == 'violator':
            df = df.rename(columns={'plate': 'violator_plate', 'state': 'violator_state'})
    return df

In [ ]:
def handle_null_values(df, bq_dataset, dimension_name):
    # Renaming for 311
    if bq_dataset == '311_illegal_parking':
      if dimension_name == 'location':
        default_values = {
          'city': 'Unspecified',
          'incident_zip': 0,
          'borough': 'Unspecified'
        }
        df.fillna(default_values, inplace=True)
        
    # Renaming for Open Parking
    elif bq_dataset == 'open_parking':
      if dimension_name == 'agency':
        default_values = {
          'issuing_agency': 'N/A'
        }
        df.fillna(default_values, inplace=True)

      elif dimension_name == 'violation':
        default_values = {
          'violation_status': 'N/A'
        }
        df.fillna(default_values, inplace=True)
        
    return df

In [ ]:
def calculate_location_attributes(df):
    precinct_to_zipcode = {
        1: 10013,
        5: 10013,
        6: 10014,
        7: 10002,
        9: 10003,
        10: 10011,
        13: 10010,
        14: 10001,
        17: 10022,
        18: 10019,
        19: 10065,
        20: 10024,
        22: 10024,
        23: 10029,
        24: 10025,
        25: 10035,
        26: 10027,
        28: 10027,
        30: 10031,
        32: 10030,
        33: 10032,
        34: 10033,
        40: 10454,
        41: 10459,
        42: 10451,
        43: 10473,
        44: 10452,
        45: 10465,
        46: 10457,
        47: 10466,
        48: 10457,
        49: 10461,
        50: 10463,
        52: 10467,
        60: 11224,
        61: 11223,
        62: 11214,
        63: 11210,
        66: 11204,
        67: 11226,
        68: 11220,
        69: 11236,
        70: 11230,
        71: 11225,
        72: 11232,
        73: 11212,
        75: 11208,
        76: 11231,
        77: 11213,
        78: 11217,
        79: 11216,
        81: 11221,
        83: 11237,
        84: 11201,
        88: 11205,
        90: 11211,
        94: 11222,
        100: 11693,
        101: 11691,
        102: 11418,
        103: 11432,
        104: 11385,
        105: 11428,
        106: 11417,
        107: 11365,
        108: 11101,
        109: 11354,
        110: 11373,
        111: 11361,
        112: 11375,
        113: 11434,
        114: 11103,
        115: 11372,
        120: 10301,
        121: 10314,
        122: 10306,
        123: 10307
    }

    zipcode_to_borough = {
        10013: "Manhattan",
        10014: "Manhattan",
        10002: "Manhattan",
        10003: "Manhattan",
        10011: "Manhattan",
        10010: "Manhattan",
        10001: "Manhattan",
        10022: "Manhattan",
        10019: "Manhattan",
        10065: "Manhattan",
        10024: "Manhattan",
        10029: "Manhattan",
        10025: "Manhattan",
        10035: "Manhattan",
        10027: "Manhattan",
        10031: "Manhattan",
        10030: "Manhattan",
        10032: "Manhattan",
        10033: "Manhattan",
        10454: "Bronx",
        10459: "Bronx",
        10451: "Bronx",
        10473: "Bronx",
        10452: "Bronx",
        10465: "Bronx",
        10457: "Bronx",
        10466: "Bronx",
        10461: "Bronx",
        10463: "Bronx",
        10467: "Bronx",
        11224: "Brooklyn",
        11223: "Brooklyn",
        11214: "Brooklyn",
        11210: "Brooklyn",
        11204: "Brooklyn",
        11226: "Brooklyn",
        11220: "Brooklyn",
        11236: "Brooklyn",
        11230: "Brooklyn",
        11225: "Brooklyn",
        11232: "Brooklyn",
        11212: "Brooklyn",
        11208: "Brooklyn",
        11231: "Brooklyn",
        11213: "Brooklyn",
        11217: "Brooklyn",
        11216: "Brooklyn",
        11221: "Brooklyn",
        11237: "Brooklyn",
        11201: "Brooklyn",
        11205: "Brooklyn",
        11211: "Brooklyn",
        11222: "Brooklyn",
        11693: "Queens",
        11691: "Queens",
        11418: "Queens",
        11432: "Queens",
        11385: "Queens",
        11428: "Queens",
        11417: "Queens",
        11365: "Queens",
        11101: "Queens",
        11354: "Queens",
        11373: "Queens",
        11361: "Queens",
        11375: "Queens",
        11434: "Queens",
        11103: "Queens",
        11372: "Queens",
        10301: "Staten Island",
        10314: "Staten Island",
        10306: "Staten Island",
        10307: "Staten Island"
    }

    df['incident_zipcode'] = df['precinct_num'].map(precinct_to_zipcode)
    df['borough'] = df['incident_zipcode'].map(zipcode_to_borough)
    return df

Create fact table for 311 illegal parking

In [ ]:
if __name__ == "__main__":
    df = pd.DataFrame
    # Create the BigQuery Client
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path_to_service_account_key_file

    # Construct a BigQuery client object
    bqclient = bigquery.Client()
    
    bq_dataset = '311_illegal_parking'
    fact_name = '311_illegal_parking'
    table_name = fact_name + '_fact'
    # Construct the full BigQuery path to the table
    fact_table_path = ".".join([gcp_project,bq_dataset,table_name])
    file_source_path = 'data/311_master.csv'

    # Load in the data file
    with open(file_source_path, 'r') as data:
            df = pd.read_csv(data)
   
    # Set all of the column names to lower case letters
    df = df.rename(columns=str.lower)

    df = rename_column(df, bq_dataset, 'complaint')
    df = dimension_lookup(dimension_name='complaint', lookup_columns=['complaint_type', 'complaint_description'], df=df)

    df = rename_column(df, bq_dataset, 'complaint_source') 
    df = dimension_lookup(dimension_name='complaint_source', lookup_columns=['complaint_source_channel'], df=df)

    df = rename_column(df, bq_dataset, 'date') 
    df['full_date'] = pd.to_datetime(df['full_date'])
    df['year'] = df['full_date'].dt.year
    df['month'] = df['full_date'].dt.month
    df['month_name'] = df['full_date'].dt.strftime('%B')
    df['day'] = df['full_date'].dt.day
    df['weekday_name'] = df['full_date'].dt.strftime('%A')
    df['full_date'] = df['full_date'].apply(lambda x: x.strftime('%Y-%m-%d'))
    df = dimension_lookup(dimension_name='date', lookup_columns=['full_date', 'year', 'month', 'month_name', 'day', 'weekday_name'], df=df)

    df = handle_null_values(df, bq_dataset, 'location')
    df = rename_column(df, bq_dataset, 'location') 
    df = dimension_lookup(dimension_name='location', lookup_columns=['borough', 'incident_city', 'incident_zipcode'], df=df)

    df = rename_column(df, bq_dataset, 'status') 
    df = dimension_lookup(dimension_name='status', lookup_columns=['status'], df=df)

    # A list of all of the surrogate keys
    # For transaction grain, also include the 'unique_key' column
    surrogate_keys=['unique_key', 'complaint_dim_id','complaint_source_dim_id','date_dim_id','location_dim_id','status_dim_id']
    
    # Remove all of the other non-surrogate key columns
    df = df[surrogate_keys]

    # See if the target table exists
    target_table_exists = bigquery_table_exists(fact_table_path, bqclient )
    # If the target table does not exist, load all of the data into a new table
    if not target_table_exists:
        build_new_table( bqclient, fact_table_path, df)
    # If the target table exists, then perform an incremental load
    if target_table_exists:
        insert_existing_table( bqclient, fact_table_path, df)

Create fact table for Open Parking

In [ ]:
if __name__ == "__main__":
    df = pd.DataFrame
    # Create the BigQuery Client
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path_to_service_account_key_file

    # Construct a BigQuery client object
    bqclient = bigquery.Client()

    bq_dataset = 'open_parking'
    fact_name = 'open_parking'
    table_name = fact_name + '_fact'
    # Construct the full BigQuery path to the table
    fact_table_path = ".".join([gcp_project,bq_dataset,table_name])
    file_source_path = 'data/open_parking_master.csv'

    # Load in the data file
    with open(file_source_path, 'r') as data:
            df = pd.read_csv(data)

    # Set all of the column names to lower case letters
    df = df.rename(columns=str.lower)

    df = rename_column(df, bq_dataset, 'agency')
    df = dimension_lookup(dimension_name='agency', lookup_columns=['agency_name'], df=df)

    df = rename_column(df, bq_dataset, 'date') 
    df['full_date'] = pd.to_datetime(df['full_date'], format='%m/%d/%Y', errors='coerce')
    df = df.dropna(subset=['full_date'], axis=0)
    df['year'] = df['full_date'].dt.year
    df['month'] = df['full_date'].dt.month
    df['month_name'] = df['full_date'].dt.strftime('%B')
    df['day'] = df['full_date'].dt.day
    df['weekday_name'] = df['full_date'].dt.strftime('%A')
    df['full_date'] = df['full_date'].apply(lambda x: x.strftime('%Y-%m-%d'))
    df = dimension_lookup(dimension_name='date', lookup_columns=['full_date', 'year', 'month', 'month_name', 'day', 'weekday_name'], df=df)

    df = rename_column(df, bq_dataset, 'location')
    df = calculate_location_attributes(df)
    df = dimension_lookup(dimension_name='location', lookup_columns=['precinct_num', 'borough', 'incident_zipcode'], df=df)

    df = handle_null_values(df, bq_dataset, 'violation')
    df = rename_column(df, bq_dataset, 'violation') 
    df = dimension_lookup(dimension_name='violation', lookup_columns=['violation_description', 'violation_status'], df=df)

    df = rename_column(df, bq_dataset, 'violator') 
    df = dimension_lookup(dimension_name='violator', lookup_columns=['violator_state', 'license_type'], df=df)

    # A list of all of the surrogate keys
    # For transaction grain, also include the 'unique_key' column
    surrogate_keys=['summons_number', 'agency_dim_id', 'location_dim_id', 'date_dim_id', 'violation_dim_id', 'violator_dim_id']
    
    # Remove all of the other non-surrogate key columns
    df = df[surrogate_keys]

    # See if the target table exists
    target_table_exists = bigquery_table_exists(fact_table_path, bqclient )
    # If the target table does not exist, load all of the data into a new table
    if not target_table_exists:
        build_new_table( bqclient, fact_table_path, df)
    # If the target table exists, then perform an incremental load
    if target_table_exists:
        insert_existing_table( bqclient, fact_table_path, df)